In [ ]:
#pip install faker

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import psycopg2
from psycopg2 import extras  

In [ ]:
from faker import Faker
fake = Faker('en_IN')

In [ ]:
conn = psycopg2.connect(database="ecomm",
                        user='sushil', password='admin123', 
                        host='34.73.147.246', port='5432'
)

conn.autocommit = True
cursor = conn.cursor()

# Generate Customer Table Data

In [ ]:
cust_fields= ['customerid','name','address','city','state','pincode','update_timestamp']

In [ ]:
states = {'Gujrat':['Ahmadabad','Surat','Vadodara'],
          'Karnataka':['Belgaum','Bengaluru','Mangalore','Mysore'],
          'Maharashtra':['Mumbai','Nagpur','Navi Mumbai','Pune'],
          'Tamil Nadu':['Chennai','Coimbatore','Madurai'],
          'Uttar Pradesh':['Agra','Greater Noida','Kanpur','Lucknow','Mathura'],
          'West Bengal':['Kolkata']}

In [ ]:
customer = pd.DataFrame(columns=cust_fields,index=range(1,1000))
Faker.seed(10)
for i in range(1,1000):
    customer['customerid'][i]=i
    customer['name'][i]=fake.name()
    customer['address'][i]=fake.street_address()    
    customer['state'][i]=fake.random_element(states.keys())
    customer['city'][i]=fake.random_element(states[customer['state'][i]])
    customer['pincode'][i]=fake.postcode()
    customer['update_timestamp'][i]=fake.date_time_this_year()


# Generate Product table data

In [ ]:
product_master_fields = ['productid','productcode','productname','sku','rate','isactive']

In [ ]:
product_initials = ['Ashirwad','Himalaya','Krishna','Shining','Lotus','APD','Glorious','Pixel','Living']
product_types = ['ata','seeds','Mix Fruits','rice','cement','bags']

In [ ]:
productname = ((a,b) for a in product_initials for b in product_types)
productlist=[]
for a,b in productname:
    productlist.append(a+ ' '+b)
print(productlist)

In [ ]:
product_master = pd.DataFrame(columns=product_master_fields,index=range(1,101))
Faker.seed(10)
for i in range(1,51):
    x = productlist[i]
    for j in range((i*2)-1,(i*2)+1):        
        product_master['productid'][j]=j
        product_master['productcode'][j]=fake.bothify('?##', letters='ABCDE')
        product_master['productname'][j]= x
        product_master['sku'][j] = str(fake.random_int(min=1, max=20)) + fake.bothify('?', letters=['KG'])
        product_master['rate'][j]=round(np.random.normal(1000,200))
        product_master['isactive'][j]=fake.boolean(chance_of_getting_true=80)

# Generate Order Details table data

In [ ]:
order_detail_fields = ['orderid','customerid','order_status_update_timestamp','order_status']

In [ ]:
orders_Received = pd.DataFrame(columns=order_detail_fields, index = range(1,20000))
orders_InProgress = pd.DataFrame(columns=order_detail_fields, index = range(1,20000))
orders_Delivered = pd.DataFrame(columns=order_detail_fields, index = range(1,20000))

In [ ]:
for i in range(1,20000):
    orders_Received['orderid'][i]=i
    orders_Received['customerid'][i]=fake.random_element(customer['customerid'])
    orders_Received['order_status_update_timestamp'][i]=fake.date_time_this_year()
    orders_Received['order_status']='Received'
    
for i in range(1,20000):
    orders_InProgress['orderid'][i]=i
    orders_InProgress['customerid'][i]=orders_Received['customerid'][i]
    orders_InProgress['order_status_update_timestamp'][i]=orders_Received['order_status_update_timestamp'][i]+timedelta(seconds=fake.random_int(min=1, max=86400))
    orders_InProgress['order_status'][i]='InProgress'
    
for i in range(1,20000):
    orders_Delivered['orderid'][i]=i
    orders_Delivered['customerid'][i]=orders_Received['customerid'][i]
    orders_Delivered['order_status_update_timestamp'][i]=orders_InProgress['order_status_update_timestamp'][i]+timedelta(seconds=fake.random_int(min=1, max=86400))
    orders_Delivered['order_status'][i]='Delivered'
    

In [ ]:
order_details_temp = orders_Received.append(orders_InProgress)
order_details = order_details_temp.append(orders_Delivered)


# Generate Order items table data

In [ ]:
order_item_fields = ['orderid','productid','quantity']

In [ ]:
order_items = pd.DataFrame(columns=order_item_fields, index = range(1,50000))

In [ ]:
for i in range(1,50000):
    order_items['orderid'][i]=fake.random_element(order_details['orderid'])
    order_items['productid'][i]=fake.random_element(product_master['productid'])
    order_items['quantity'][i]=fake.random_int(min=1, max=20)

In [ ]:
order_items = order_items.drop_duplicates(subset=['orderid', 'productid'],keep='first')
#order_items.sort_values(by=['orderid','productid'])

# Load DFs into Postgres tables

In [ ]:
table = 'customer_master'
df_columns = ",".join(cust_fields)
values = "VALUES({})".format(",".join(["%s" for _ in cust_fields])) 
insert_stmt = "INSERT INTO {} ({}) {}".format(table,df_columns,values)

psycopg2.extras.execute_batch(cursor, insert_stmt, customer.values)

conn.commit()

In [ ]:
table = 'product_master'
df_columns = ",".join(product_master_fields)
values = "VALUES({})".format(",".join(["%s" for _ in product_master_fields])) 
insert_stmt = "INSERT INTO {} ({}) {}".format(table,df_columns,values)

psycopg2.extras.execute_batch(cursor, insert_stmt, product_master.values)

conn.commit()

In [ ]:
table = 'order_details'
df_columns = ",".join(order_detail_fields)
values = "VALUES({})".format(",".join(["%s" for _ in order_detail_fields])) 
insert_stmt = "INSERT INTO {} ({}) {}".format(table,df_columns,values)

psycopg2.extras.execute_batch(cursor, insert_stmt, order_details.values)

conn.commit()

In [ ]:
table = 'order_items'
df_columns = ",".join(order_item_fields)
values = "VALUES({})".format(",".join(["%s" for _ in order_item_fields])) 
insert_stmt = "INSERT INTO {} ({}) {}".format(table,df_columns,values)

psycopg2.extras.execute_batch(cursor, insert_stmt, order_items.values)

conn.commit()

# Validate

In [ ]:
sql1='''select count(1) from customer_master limit 50;'''

In [ ]:
sql2='''select count(1) from product_master  limit 50;'''

In [ ]:
sql3='''select count(1) from order_details  limit 50;'''

In [ ]:
sql4='''select count(1) from order_items limit 50;'''

In [ ]:
cursor.execute(sql1)
for i in cursor.fetchall():
    print(i)

In [ ]:
#Purge all OLTP tables
for tbl in ('order_details','order_items','customer_master','product_master'):
    sql=f'''delete from {tbl};'''
    cursor.execute(sql)

In [ ]:
#ETL Logs table
sql1='''Select * from etlextractlog;'''
cursor.execute(sql1)
for i in cursor.fetchall():
    print(i)